# «Модель прогнозирования стоимости жилья для агентства недвижимости»

### ПОСТАНОВКА ЗАДАЧИ

Агентство недвижимости столкнулось с проблемой — риелторы тратят слишком много времени на сортировку объявлений и поиск выгодных предложений. Поэтому скорость их реакции и качество анализа не дотягивают до уровня конкурентов. Это сказывается на финансовых показателях агентства.

Типовая задача из области Data Science, основанная на данных о недвижимости в США.

**Бизнес-задача:** улучшить финансовые показатели компании за счет уменьшения времени на обработку информации о недвижимости у риелторов.

**Техническая задача для вас как для специалиста в Data Science:** разработать модель машинного обучения, которая поможет обрабатывать объявления и увеличит число сделок и прибыль агентства.

**Основные цели проекта:**
1. Провести разведывательный анализ и очистку исходных данных.
2. Выделить наиболее значимые факторы, влияющие на стоимость недвижимости.
3. Построить модель для прогнозирования стоимости недвижимости.
4. Разработать небольшой веб-сервис, на вход которому поступают данные о некоторой выставленной на продажу недвижимости, а сервис прогнозирует его стоимость.

### ЗНАКОМСТВО С ДАННЫМИ

In [1]:
# импортируем необходимые библиотеки
import pandas as pd
import numpy as np

Познакомимся с данными:

In [2]:
#прочитаем данные из файла
df = pd.read_csv('data.csv', sep = ',') # прочитаем файл с данными

In [3]:
#проверим, что данные выгрузились корректно, для этого выведем первые две строки датафрейма
display('Таблица данных недвижимости',df.head(2))

'Таблица данных недвижимости'

,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
0,Active,NaN,Single Family Home,240 Heather Ln,3.5,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",Gas Logs,Southern Pines,"[{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...",2900,28387,4,NC,NaN,NaN,NaN,611019,"$418,000"
1,for sale,NaN,single-family home,12911 E Heroy Ave,3 Baths,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",NaN,Spokane Valley,"[{'rating': ['4/10', 'None/10', '4/10'], 'data...","1,947 sqft",99216,3 Beds,WA,2.0,NaN,NaN,201916904,"$310,000"


Итак, в нашем распоряжении более чем 300 тыс. наблюдений и 18 признаков:

* 'propertyType' — тип объекта недвижимости;
* 'street' — адрес объекта;
* 'baths' — количество ванных комнат;
* 'homeFacts' — сведения о строительстве объекта (содержит несколько типов сведений, влияющих на оценку объекта);
* 'fireplace' — наличие камина;
* 'city' — город;
* 'schools' — сведения о школах в районе;
* 'sqft' — площадь в футах;
* 'zipcode' — почтовый индекс;
* 'beds' — количество спален;
* 'state' — штат;
* 'stories' — количество этажей;
* 'mls-id' и 'MlsId' — идентификатор MLS (Multiple Listing Service, система мультилистинга);
* 'target' — цена объекта недвижимости (целевой признак, который необходимо спрогнозировать).

Два признака в нашем наборе данных являются словарями: schools и homeFacts. Для дальнейшей обработки проведем их десериализацию. 

Из словаря schools выделим наиболее нам интересные признаки: рейтинг школ (sch_rating) и расстояние до школ (sch_dist).

Из словаря homeFacts наиболее информативными для нашей задачи считаю признаки: год постройки недвижимости (yearBuilt), год реконструкции недвижимости (remodeledYear), информация об отоплении (heating), информация о системе охлаждения (cooling), информация о парковке (parking),  цена за квадратный фут (price_sqft).

In [4]:
# Функция десериализации строки со словарем значений
def deserialize_home(s):
    """Функция на вход получает строку с сериализованным словарем.
    
    Функция возвращат значения словаря, который находится
    в сериализованном виде в строке s.
    """

    # Десериализуем строку в словарь и сразу сохраним его единственое значение
    result = eval(s)['atAGlanceFacts'][:6]

    # В единственном значении словаря хранится список из 7 элементов,
    # отберем из них 6
    # каждый из них представлен двумя словарями: factValue и factLabel
    # С помощью дополнения списка сохраним в список только factValue
    # и вернем их, а там уже присвоим им названия
    return pd.Series([r['factValue'] for r in result])

In [5]:
# Воспользуемся написанной функций, чтобы создать новые признаки из homeFacts
df[['yearBuilt', 'remodeledYear', 'heating', 'cooling',
    'parking', 'price_sqft']] = df['homeFacts'].apply(deserialize_home)

In [6]:
# Функция десериализации признака school
def deserialize_school(s):
    """Функция принимает на вход строку с сериализованным словарем.
    
    Функция возвращает значения словаря, который находится
    в сериализованном виде в строке s.
    """

    # Десериализуем строку в словарь и сразу сохраним его единственое значение
    result = eval(s)[0]

    # В единственном значении словаря хранится словарь из 3 элементов,
    # каждый из которых представлен списками (за исключением второго - data,
    # у которого два списка обернуты в словарь).
    # Вернем рейтинги и расстояние до школ
    rating = result['rating']
    distance = result['data']['Distance']

    # Возвращаем списки в виде объекта Series
    return pd.Series([rating, distance])

In [7]:
# Десериализуем признак schools
df[['sch_rating', 'sch_dist']] = df['schools'].apply(deserialize_school)

In [8]:
#посмотрим на таблицу, с которой нам предстоит работать
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377185 entries, 0 to 377184
Data columns (total 26 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   status         337267 non-null  object
 1   private pool   4181 non-null    object
 2   propertyType   342452 non-null  object
 3   street         377183 non-null  object
 4   baths          270847 non-null  object
 5   homeFacts      377185 non-null  object
 6   fireplace      103114 non-null  object
 7   city           377151 non-null  object
 8   schools        377185 non-null  object
 9   sqft           336608 non-null  object
 10  zipcode        377185 non-null  object
 11  beds           285903 non-null  object
 12  state          377185 non-null  object
 13  stories        226469 non-null  object
 14  mls-id         24942 non-null   object
 15  PrivatePool    40311 non-null   object
 16  MlsId          310305 non-null  object
 17  target         374704 non-null  object
 18  year

Видим, что данные не совсем полные. В четырех признаках из 18 пропуски на первый взгляд отсутствуют: расстоянии до школы, рейтинге школ, почтовые индексы, штат.

Более детально поработаем с пропусками на этапе обработки и очистки данных.

In [9]:
#выгрузим таблицу с данными для дальнейшей обработки в другом ноутбуке
df.to_csv('full_data.csv',index=False)